# Optical Music Recognition
**By: Sarah Alabdulwahab & Asma Althakafi**
> Optical Music Recognition (OMR) is software that essentially teaches the machine to read sheet music. The aim of this project is to develop a sheet music reader which will classify each music note then produce an audio file of the music it read.

In [1]:
#supress warnings
import warnings
warnings.filterwarnings('ignore')

from tqdm import tqdm
import numpy as np
from pre_processing import *
from scipy.ndimage import binary_fill_holes
from skimage.morphology import thin

#Predefined models from github.com/aashrafh
from commonfunctions import *
from staff import coordinator
from segmenter import Segmenter
from connected_componentes import *
from fit import predict
from box import Box

In [2]:
label_map = {0:{0: 'N0'}, 1:{0:'b2',1:'a2'}, 2:{0:'g2',1:'f2'}, 3:{0:'e2',1:'d2'}, 4:{0:'c2',1:'b1'}, 5:{0:'a1',1:'g1'}
             , 6:{0:'f1',1:'e1'}, 7:{0:'d1',1:'c1'}}

In [3]:
def read_all_images(num_of_images):
    all_images_list = []
    for i in range(num_of_images):
        path = f'Notes/{i+1}.png'
        img = gray_img(io.imread(path))
        all_images_list.append(get_thresholded(img, threshold_otsu(img)))
    return all_images_list

In [4]:
def segmenting(img):
    segmenter = Segmenter(get_thresholded(img, threshold_otsu(img))) #binary image
    imgs_with_staff = segmenter.regions_with_staff
    imgs_spacing, imgs_rows, coord_imgs  = [], [], []
    for i, img in enumerate(imgs_with_staff):
        spacing, rows, no_staff_img = coordinator(img,IsHorizontal(img))
        imgs_rows.append(rows)
        imgs_spacing.append(spacing)
        coord_imgs.append(no_staff_img)
    return segmenter, imgs_spacing, imgs_rows, coord_imgs, imgs_with_staff

In [5]:
def estim(c, idx, imgs_spacing, imgs_rows):
    spacing = imgs_spacing[idx]
    rows = imgs_rows[idx]
    margin = 1+(spacing/4)
    for index,line in enumerate (rows):
        if c >= line - margin and c <= line + margin:
            return index+1, 0
        elif c >= line + margin and c <= line + 3*margin:
            return index+1, 1
    return 0, 0 

In [6]:
def filter_beams(prims, prim_with_staff, bounds):
    n_bounds = []
    n_prims = []
    n_prim_with_staff = []
    for i, prim in enumerate(prims):
        if prim.shape[1] >= 2*prim.shape[0]:
            continue
        else:
            n_bounds.append(bounds[i])
            n_prims.append(prims[i])
            n_prim_with_staff.append(prim_with_staff[i])
    return n_prims, n_prim_with_staff, n_bounds

In [7]:
def get_labeled_data(img):
    labels_list, images_list = [], []
    segmenter, imgs_spacing, imgs_rows, coord_imgs, imgs_with_staff = segmenting(img)
    black_names = ['4', '8', '8_b_n', '8_b_r', '16', '16_b_n', '16_b_r', '32', '32_b_n', '32_b_r', 'a_4', 
                   'a_8', 'a_16', 'a_32', 'chord']
    disk_size = segmenter.most_common / 4
    for i, img in enumerate(coord_imgs):
        primitives, prim_with_staff, boundary = get_connected_components(img, imgs_with_staff[i])
        for j, prim in enumerate(primitives):
            prim = binary_opening(prim, square(segmenter.most_common-imgs_spacing[i]))
            label = predict((255*(1 - prim)).astype(np.uint8))[0]
            if label in black_names:
                test_img = binary_dilation(np.copy(prim_with_staff[j]), disk(disk_size))
                comps, comp_w_staff, bounds = get_connected_components(test_img, prim_with_staff[j])
                comps, comp_w_staff, bounds = filter_beams(comps, comp_w_staff, bounds)
                bounds = [np.array(bound)+disk_size-2 for bound in bounds]
                if len(bounds) <= 1 or label in ['8_b_n', '8_b_r', '16_b_n', '16_b_r', '32_b_n', '32_b_r']:
                    for bbox in bounds:
                        line_idx, p = estim(int(bbox[2]+boundary[j][0]), i, imgs_spacing, imgs_rows)
                        labels_list.append(label_map[line_idx][p])
                        images_list.append(prim_with_staff[j])
            elif label in ['2', 'a_2']:
                head_img = binary_closing(1-binary_fill_holes(1-prim), disk(disk_size))
                comps, comp_w_staff, bounds = get_connected_components(head_img, prim_with_staff[j])
                for bbox in bounds:
                    line_idx, p = estim(int(bbox[2]+boundary[j][0]), i, imgs_spacing, imgs_rows)
                    labels_list.append(label_map[line_idx][p])
                    images_list.append(prim_with_staff[j])
            elif label in ['1', 'a_1']:
                line_idx, p = estim(int(boundary[j][2]), i, imgs_spacing, imgs_rows)
                labels_list.append(label_map[line_idx][p])
                images_list.append(prim_with_staff[j])
    return labels_list, images_list

## Data Collection

In [8]:
#testing on small data
all_images = read_all_images(50)
len(all_images)

50

In [9]:
all_images_labels, all_images_arr, all_images_ids =[], [], []
for i, image in tqdm(enumerate(all_images)):
    labels_list, images_list = get_labeled_data(image)
    all_images_labels += labels_list
    all_images_arr += images_list
    all_images_ids += [i] * len(labels_list)

50it [05:35,  6.72s/it]


In [10]:
#view some labels
all_images_labels[:10]

['b2', 'e2', 'b2', 'N0', 'b2', 'a2', 'a2', 'a2', 'g2', 'a2']

In [11]:
#check that all lengths match
len(all_images_labels) == len(all_images_arr) == len(all_images_ids)

True

## Store The Data

In [12]:
import pandas as pd

In [13]:
df = pd.DataFrame(data= {'ID':all_images_ids, 'img': all_images_arr, 'label': all_images_labels})
df.head()

,ID,img,label
0,0,"[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...",b2
1,0,"[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...",e2
2,0,"[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...",b2
3,0,"[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...",N0
4,0,"[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...",b2


In [14]:
df.shape

(419, 3)

In [15]:
unique_labels = df.label.unique()
for label in unique_labels:
    df[label] = 0
df.head()

,ID,img,label,b2,e2,N0,a2,g2,d2,c2,b1,e1,g1,f2,f1,a1,d1,c1
0,0,"[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...",b2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,"[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...",e2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,"[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...",b2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,"[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...",N0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,"[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...",b2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [16]:
for i, label in enumerate(df.label):
    df[label][i] = 1
df.head()

,ID,img,label,b2,e2,N0,a2,g2,d2,c2,b1,e1,g1,f2,f1,a1,d1,c1
0,0,"[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...",b2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,"[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...",e2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,"[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...",b2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,"[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...",N0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,0,"[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...",b2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Unify the shape

In [17]:
def unify_shape(df):
    rows, cols = [], []
    for image in df.img:
        rows.append(image.shape[0])
        cols.append(image.shape[1])
    
    rows = max(rows)
    cols = max(cols)
    for i in range(len(df)):
        add_rows = np.ones((rows - df['img'][i].shape[0], df['img'][i].shape[1]), dtype = int)
        df['img'][i] = np.vstack((df['img'][i], add_rows))

        add_cols = np.ones((rows, cols - df['img'][i].shape[1]), dtype = int)
        df['img'][i] = np.hstack((df['img'][i], add_cols))
    return df, rows, cols

In [18]:
df, rows, cols = unify_shape(df)

In [19]:
#checking shapes
for x in df.img[:5]:
    print(x.shape)

(139, 171)
(139, 171)
(139, 171)
(139, 171)
(139, 171)


## Training

In [20]:
from sklearn import preprocessing
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, GlobalAveragePooling2D, InputLayer

In [21]:
X, y = np.asarray(df['img']), np.asarray(df.drop(columns=['ID','img','label']))

In [22]:
X.shape

(419,)

In [23]:
#reshaping
def reshape_x(X):
    return np.asarray([x.reshape((rows, cols, 1)) for x in X])

In [24]:
X = reshape_x(X)
X.shape

(419, 139, 171, 1)

In [25]:
y.shape

(419, 15)

In [26]:
X_train, X_test, y_train, y_test = (train_test_split(X, y, test_size = .2, random_state = 42))

In [27]:
NN = Sequential()
NN.add(InputLayer(input_shape=X.shape[1:]))
NN.add(Conv2D(filters=10, kernel_size=3, activation='relu', padding='same'))
NN.add(MaxPooling2D())
NN.add(Conv2D(filters=20, kernel_size=3, activation='relu', padding='same'))
NN.add(Conv2D(filters=16, kernel_size=3, activation='relu', padding='same'))
NN.add(MaxPooling2D())
NN.add(Conv2D(filters=30, kernel_size=3, activation='relu', padding='same'))
NN.add(GlobalAveragePooling2D())
NN.add(Dense(df.label.nunique() , activation='softmax'))

#compile is back propagation to edit the weight
NN.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
NN.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 139, 171, 10)      100       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 69, 85, 10)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 69, 85, 20)        1820      
                                                                 
 conv2d_2 (Conv2D)           (None, 69, 85, 16)        2896      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 34, 42, 16)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 34, 42, 30)        4

In [28]:
NN.fit(X_train, y_train, epochs=150, verbose=1, validation_split=0.25);

Epoch 1/150
8/8 [==============================] - 6s 432ms/step - loss: 2.6985 - accuracy: 0.0677 - val_loss: 2.6761 - val_accuracy: 0.2024
Epoch 2/150
8/8 [==============================] - 3s 352ms/step - loss: 2.6530 - accuracy: 0.1474 - val_loss: 2.5968 - val_accuracy: 0.1548
Epoch 3/150
8/8 [==============================] - 3s 355ms/step - loss: 2.5823 - accuracy: 0.1633 - val_loss: 2.5217 - val_accuracy: 0.1548
Epoch 4/150
8/8 [==============================] - 3s 412ms/step - loss: 2.5624 - accuracy: 0.1633 - val_loss: 2.4955 - val_accuracy: 0.1548
Epoch 5/150
8/8 [==============================] - 4s 446ms/step - loss: 2.5372 - accuracy: 0.1594 - val_loss: 2.4863 - val_accuracy: 0.1667
Epoch 6/150
8/8 [==============================] - 3s 439ms/step - loss: 2.5347 - accuracy: 0.1554 - val_loss: 2.4631 - val_accuracy: 0.1667
Epoch 7/150
8/8 [==============================] - 3s 416ms/step - loss: 2.5281 - accuracy: 0.1554 - val_loss: 2.4318 - val_accuracy: 0.1667
Epoch 8/150
8

Epoch 59/150
8/8 [==============================] - 3s 425ms/step - loss: 2.3439 - accuracy: 0.1673 - val_loss: 2.4622 - val_accuracy: 0.1190
Epoch 60/150
8/8 [==============================] - 3s 423ms/step - loss: 2.3236 - accuracy: 0.2112 - val_loss: 2.4847 - val_accuracy: 0.1071
Epoch 61/150
8/8 [==============================] - 3s 415ms/step - loss: 2.3243 - accuracy: 0.1952 - val_loss: 2.4587 - val_accuracy: 0.1190
Epoch 62/150
8/8 [==============================] - 3s 433ms/step - loss: 2.3228 - accuracy: 0.1912 - val_loss: 2.4877 - val_accuracy: 0.1905
Epoch 63/150
8/8 [==============================] - 3s 408ms/step - loss: 2.3149 - accuracy: 0.1793 - val_loss: 2.4635 - val_accuracy: 0.1190
Epoch 64/150
8/8 [==============================] - 3s 418ms/step - loss: 2.3178 - accuracy: 0.1952 - val_loss: 2.4680 - val_accuracy: 0.1071
Epoch 65/150
8/8 [==============================] - 3s 427ms/step - loss: 2.3171 - accuracy: 0.2072 - val_loss: 2.4616 - val_accuracy: 0.1310
Epoch 

8/8 [==============================] - 3s 424ms/step - loss: 2.0317 - accuracy: 0.2629 - val_loss: 2.4945 - val_accuracy: 0.1310
Epoch 117/150
8/8 [==============================] - 3s 415ms/step - loss: 2.0168 - accuracy: 0.2709 - val_loss: 2.4454 - val_accuracy: 0.1905
Epoch 118/150
8/8 [==============================] - 3s 432ms/step - loss: 1.9934 - accuracy: 0.2749 - val_loss: 2.4863 - val_accuracy: 0.1905
Epoch 119/150
8/8 [==============================] - 3s 415ms/step - loss: 1.9865 - accuracy: 0.2829 - val_loss: 2.4826 - val_accuracy: 0.1667
Epoch 120/150
8/8 [==============================] - 3s 406ms/step - loss: 1.9973 - accuracy: 0.2550 - val_loss: 2.6220 - val_accuracy: 0.1548
Epoch 121/150
8/8 [==============================] - 3s 405ms/step - loss: 1.9996 - accuracy: 0.2829 - val_loss: 2.4647 - val_accuracy: 0.1786
Epoch 122/150
8/8 [==============================] - 3s 406ms/step - loss: 1.9722 - accuracy: 0.2789 - val_loss: 2.4875 - val_accuracy: 0.2024
Epoch 123/150